# Mental Health Chatbot

## Goal


## Imports

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
import datetime
import sys
import os
import wandb
import gradio as gr
import psycopg2
from dotenv import load_dotenv
from datasets import load_dataset, concatenate_datasets
load_dotenv()
NEON_PG_CONNECTION_URL = os.environ['NEON_PG_CONNECTION_URL']

## DB, Wandb Connection & Setup

In [37]:
try:
    connection = psycopg2.connect(NEON_PG_CONNECTION_URL)
    connection.autocommit = True
    print("Connected to Neon Postgres!")
except Exception as e:
    print("Cannot connect to Neon Postgres:", e)

cursor = connection.cursor()

wandb.init(project="mental_health_chatbot")

Connected to Neon Postgres!


## Data Pre-Processing

In [46]:
# Load the dataframes
df1 = pd.read_json("hf://datasets/Amod/mental_health_counseling_conversations/combined_dataset.json", lines=True)
df2 = pd.read_parquet("hf://datasets/mpingale/mental-health-chat-dataset/data/train-00000-of-00001-991edb316b3098d3.parquet")

# Change column name "Context" to "Question" and "Response" to "Answer"
df1 = df1.rename(columns={"Context": "Question", "Response": "Answer"})
# Remove the "questionID" column from df2
df2 = df2.rename(columns={"questionText": "Question", "answerText": "Answer"})
df2 = df2.drop(columns=["questionID", "questionTitle", "questionLink", "therapistInfo", "topic", "therapistURL", "upvotes", "views", "text"])

# combine the two dataframes and add an index column
df = pd.concat([df1, df2], ignore_index=True)

# Print the first 5 rows of the dataframe
print(df.head())

# Use try-except blocks. First, the table if it already exists. Create a new table
try:
    cursor.execute(
        """
        CREATE TABLE mental_health_qa (
            id BIGSERIAL PRIMARY KEY,
            question TEXT,
            answer TEXT,
            vector VECTOR(384)
        );
    """
    )
    print("Created table emails.")
except Exception as e:
    print("Cannot create table emails:", e)


                                            Question  \
0  I'm going through some things with my feelings...   
1  I'm going through some things with my feelings...   
2  I'm going through some things with my feelings...   
3  I'm going through some things with my feelings...   
4  I'm going through some things with my feelings...   

                                              Answer  
0  If everyone thinks you're worthless, then mayb...  
1  Hello, and thank you for your question and see...  
2  First thing I'd suggest is getting the sleep y...  
3  Therapy is essential for those that are feelin...  
4  I first want to let you know that you are not ...  
Cannot insert data into mental_health_chatbot table: terminating connection due to administrator command
SSL connection has been closed unexpectedly

